**using this there is no need to reload your packages every time when you will make any improvement to them**

In [1]:
%load_ext autoreload

In [2]:
autoreload 2

**import necessary libraries**

In [3]:
import numpy as np
import pandas as pd 
import sys
import warnings

**set warnings options**

In [22]:
warnings.filterwarnings('ignore')

#### to have ease in importing modules

In [5]:
sys.path.insert(0, '/Users/mjasiecz/PycharmProjects/new_offer_success_predictor/src')

**set pandas options**

In [6]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 200

**balancing dataset**

In [7]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

Using TensorFlow backend.


**import packages for manage and preprocess data**

In [8]:
from data.data_manager import DataManager
from data.data_processor import DataProcessor, TestDataProcessor
from ml_preprocessing.categorical_encoders import LeaveOneOutEncoder

**load data pipeline**

In [9]:
DM = DataManager()
train_df, test_df = DM.load_train_test()

In [10]:
train_df.head()

,accepted,offer_class,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
name,,,,,,,,,,,,,,
8550AB469CB2445,no,Premium,female,NaN,1.0,2.0,2678,1524.58,8CA,C,0F2A2,303.973257,7207.0,B
07355EE27DD1493,no,High,male,32.0,0.0,0.0,244360,1300.00,9DD,S,5F15A,64.075055,3675.0,A
034E73A251554F0,yes,Premium,female,NaN,1.0,1.0,370365,1550.00,517,Q,AC578,108.581750,4750.0,B
0AF961B4AC7A439,no,Premium,male,NaN,0.0,1.0,2652,722.92,9CB,C,8A8CC,155.495957,1401.0,A
8535BBCA690A4AE,yes,Premium,male,NaN,0.0,1.0,1601,5649.58,6FF,S,42EFB,121.371033,2106.0,B


In [11]:
train_df.shape

(1031, 14)

In [12]:
test_df.head()

,accepted,offer_class,gender,age,phone_calls,emails,customer_code,salary,offer_code,customer_type,number,offer_value,estimated_expenses,center
name,,,,,,,,,,,,,,
DA3BAEB8BF604EB,no,Premium,male,NaN,0.0,1.0,Fa 265302,731.25,99F,S,D6706,143.108044,7638.0,A
AEF3DE08DFED4E0,no,Premium,male,31.0,0.0,0.0,347063,777.50,405,S,NaN,63.637416,5784.0,A
B621BB29484E46D,no,Premium,male,25.0,0.0,1.0,349250,789.58,954,S,7E9CB,210.204552,4605.0,B
2D0945802F92423,no,Premium,female,18.0,0.0,1.0,365226,675.00,D7D,Q,6361F,131.110924,4459.0,B
640ABFC7E49B403,no,Medium,male,56.0,0.0,1.0,17764,3069.58,BAF,C,63C3B,180.019199,4396.0,B


In [13]:
test_df.shape

(258, 14)

**ml data preprocessing**

In [14]:
DP = DataProcessor(train_df=train_df)
processed_train_df = DP.perform_initial_features_engineering()
TDP = TestDataProcessor(not_processed_train_df=train_df,
                        processed_train_df=processed_train_df,
                        test_df=test_df,
                        sneaky_peaky=True)
processed_test_df = TDP.perform_initial_features_engineering()

Imputing row 1/1031 with 1 missing, elapsed time: 0.221
Imputing row 1/1289 with 0 missing, elapsed time: 0.304
Imputing row 1033/1289 with 0 missing, elapsed time: 0.307


In [15]:
processed_train_df.head()

,cc_len,cc_startswith,log_salary,log_age_knn,log_estimated_expenses_knn,log_offer_value_knn,scaled_salary,scaled_age_knn,scaled_estimated_expenses_knn,scaled_offer_value_knn,nan_age,not_nan_age,target,offer_class,gender,age,phone_calls,emails,salary,customer_type,center,age_knn,estimated_expenses_knn,offer_value_knn
name,,,,,,,,,,,,,,,,,,,,,,,,
8550AB469CB2445,ELSE,ELSE,7.329474,3.401259,8.882808,5.716940,-0.345530,-0.297159,1.362250,3.035567,True,False,False,Premium,female,NaN,1,2,1524.58,C,B,30.001848,7207.0,303.973257
07355EE27DD1493,ELSE,ELSE,7.170120,3.465736,8.209308,4.160055,-0.389281,-0.111655,-0.492006,-1.109564,False,True,False,High,male,32.0,0,0,1300.00,S,A,32.000000,3675.0,64.075055
034E73A251554F0,ELSE,ELSE,7.346010,3.177471,8.465900,4.687503,-0.340578,-0.855654,0.072355,-0.340546,True,False,True,Premium,female,NaN,1,1,1550.00,Q,B,23.986022,4750.0,108.581750
0AF961B4AC7A439,ELSE,ELSE,6.583299,3.323884,7.244942,5.046620,-0.501704,-0.504546,-1.685828,0.470071,True,False,False,Premium,male,NaN,0,1,722.92,C,A,27.767979,1401.0,155.495957
8535BBCA690A4AE,ELSE,ELSE,8.639336,3.405772,7.652546,4.798852,0.458072,-0.284561,-1.315712,-0.119564,True,False,True,Premium,male,NaN,0,1,5649.58,S,B,30.137541,2106.0,121.371033


In [16]:
processed_test_df.head()

,cc_len,cc_startswith,log_salary,log_age_knn,log_estimated_expenses_knn,log_offer_value_knn,scaled_salary,scaled_age_knn,scaled_estimated_expenses_knn,scaled_offer_value_knn,nan_age,not_nan_age,target,age,center,customer_type,emails,gender,offer_class,phone_calls,salary,age_knn,estimated_expenses_knn,offer_value_knn
name,,,,,,,,,,,,,,,,,,,,,,,,
DA3BAEB8BF604EB,ELSE,ELSE,6.594755,3.489576,8.940891,4.963600,-0.501839,-0.035049,1.603528,0.246935,True,False,False,NaN,A,S,1,male,Premium,0,731.25,32.772049,7638.0,143.108044
AEF3DE08DFED4E0,ELSE,ELSE,6.656084,3.433987,8.662851,4.153202,-0.492946,-0.197055,0.631794,-1.130746,False,True,False,31.0,A,S,0,male,Premium,0,777.50,31.000000,5784.0,63.637416
B621BB29484E46D,ELSE,ELSE,6.671501,3.218876,8.434898,5.348081,-0.490623,-0.745594,0.013846,1.410101,False,True,False,25.0,B,S,1,male,Premium,0,789.58,25.000000,4605.0,210.204552
2D0945802F92423,ELSE,ELSE,6.514713,2.890372,8.402680,4.876044,-0.512654,-1.385557,-0.062677,0.038956,False,True,False,18.0,B,Q,1,female,Premium,0,675.00,18.000000,4459.0,131.110924
640ABFC7E49B403,58,ELSE,8.029296,4.025352,8.388450,5.193064,-0.052226,2.088525,-0.095697,0.886816,False,True,False,56.0,B,C,1,male,Medium,0,3069.58,56.000000,4396.0,180.019199


**verify if there is no mistake during preprocessing**

In [17]:
assert not set(processed_train_df.index).intersection(set(processed_test_df.index))

**left to process: categoricalencoding and age binning approach**

In [18]:
processed_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1031 entries, 8550AB469CB2445 to 358EEC4160A4478
Data columns (total 24 columns):
cc_len                           1031 non-null object
cc_startswith                    1031 non-null object
log_salary                       1031 non-null float64
log_age_knn                      1031 non-null float64
log_estimated_expenses_knn       1031 non-null float64
log_offer_value_knn              1031 non-null float64
scaled_salary                    1031 non-null float64
scaled_age_knn                   1031 non-null float64
scaled_estimated_expenses_knn    1031 non-null float64
scaled_offer_value_knn           1031 non-null float64
nan_age                          1031 non-null bool
not_nan_age                      1031 non-null bool
target                           1031 non-null bool
offer_class                      1031 non-null object
gender                           1031 non-null object
age                              688 non-null float64
phone_c

In [19]:
processed_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258 entries, DA3BAEB8BF604EB to 2E27EC78F50943B
Data columns (total 24 columns):
cc_len                           258 non-null object
cc_startswith                    258 non-null object
log_salary                       258 non-null float64
log_age_knn                      258 non-null float64
log_estimated_expenses_knn       258 non-null float64
log_offer_value_knn              258 non-null float64
scaled_salary                    258 non-null float64
scaled_age_knn                   258 non-null float64
scaled_estimated_expenses_knn    258 non-null float64
scaled_offer_value_knn           258 non-null float64
nan_age                          258 non-null bool
not_nan_age                      258 non-null bool
target                           258 non-null bool
age                              189 non-null float64
center                           258 non-null object
customer_type                    258 non-null object
emails                 

**sklearn modelling**

In [20]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

**sklearn models which have predict_proba method**

In [23]:
from sklearn.utils.testing import all_estimators

estimators = all_estimators()

for name, class_ in estimators:
    if hasattr(class_, 'predict_proba'):
        print(name)

AdaBoostClassifier
BaggingClassifier
BayesianGaussianMixture
BernoulliNB
CalibratedClassifierCV
ClassifierChain
ComplementNB
DecisionTreeClassifier
DummyClassifier
ExtraTreeClassifier
ExtraTreesClassifier
GaussianMixture
GaussianNB
GaussianProcessClassifier
GradientBoostingClassifier
GridSearchCV
HistGradientBoostingClassifier
KNeighborsClassifier
LabelPropagation
LabelSpreading
LinearDiscriminantAnalysis
LogisticRegression
LogisticRegressionCV
MLPClassifier
MultiOutputClassifier
MultinomialNB
NuSVC
OneVsRestClassifier
Pipeline
QuadraticDiscriminantAnalysis
RFE
RFECV
RandomForestClassifier
RandomizedSearchCV
SGDClassifier
SVC
VotingClassifier
_BinaryGaussianProcessClassifierLaplace
_ConstantPredictor


In [618]:
enc = LeaveOneOutEncoder(train_df=processed_train_df,
                         test_df=processed_test_df,
                         columns_to_encode=['offer_class',
                                            'gender',
                                            'customer_type',
                                            #'emails',
                                            'center',
                                            'phone_calls',
                                            'cc_len',
                                            'cc_startswith'],
                         target_column='target',
                         random_state=42,
                         mean=1,
                         std=0.05)

In [368]:
train, test = enc.fit()

In [64]:
train.head(1)

,cc_len,cc_startswith,log_salary,log_age_knn,log_estimated_expenses_knn,log_offer_value_knn,scaled_salary,scaled_age_knn,scaled_estimated_expenses_knn,scaled_offer_value_knn,nan_age,not_nan_age,target,offer_class,gender,age,phone_calls,emails,salary,customer_type,center,age_knn,estimated_expenses_knn,offer_value_knn,encoded_offer_class,encoded_gender,encoded_customer_type,encoded_emails,encoded_center,encoded_phone_calls,encoded_cc_len,encoded_cc_startswith
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8550AB469CB2445,ELSE,ELSE,7.329474,3.401259,8.882808,5.71694,-0.34553,-0.297159,1.36225,3.035567,True,False,False,Premium,female,NaN,1,2,1524.58,C,B,30.001848,7207.0,303.973257,0.263477,0.744563,0.59357,0.521735,0.400552,0.530646,0.324854,0.376086


In [65]:
test.head(1)

,cc_len,cc_startswith,log_salary,log_age_knn,log_estimated_expenses_knn,log_offer_value_knn,scaled_salary,scaled_age_knn,scaled_estimated_expenses_knn,scaled_offer_value_knn,nan_age,not_nan_age,target,age,center,customer_type,emails,gender,offer_class,phone_calls,salary,age_knn,estimated_expenses_knn,offer_value_knn,encoded_offer_class,encoded_gender,encoded_customer_type,encoded_emails,encoded_center,encoded_phone_calls,encoded_cc_len,encoded_cc_startswith
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DA3BAEB8BF604EB,ELSE,ELSE,6.594755,3.489576,8.940891,4.9636,-0.501839,-0.035049,1.603528,0.246935,True,False,False,NaN,A,S,1,male,Premium,0,731.25,32.772049,7638.0,143.108044,0.256637,0.191617,0.31882,0.347072,0.365801,0.341429,0.316583,0.366552


In [435]:
#kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf = KFold(n_splits=5, shuffle=False, random_state=None) #to have each sample exactly once in validation set
splits = kf.split(processed_train_df)

In [619]:
processed_train_df = pd.concat([processed_train_df, pd.get_dummies(processed_train_df['emails'])], axis=1).rename(columns=email_ohe_names)

In [620]:
processed_test_df = pd.concat([processed_test_df, pd.get_dummies(processed_test_df['emails'])], axis=1).rename(columns=email_ohe_names).head(1)

In [621]:
X = processed_train_df.reset_index().drop(columns='name')
y = processed_train_df[['target']].reset_index().drop(columns='name')

In [622]:
len(X.columns)

30

In [623]:
# processed_train_df[['target', 'emails']].groupby(['emails', 'target']).size()

In [624]:
email_ohe_names = {0: '0_emails',
                   1: '1_email',
                   2: '2_emails',
                   3: '3_emails',
                   4: '4_emails',
                   5: '5_emails'
                  }

**primary set of chosen features**

In [625]:
X.drop(columns=['salary',
                'scaled_salary',
                'scaled_age_knn',
                'scaled_estimated_expenses_knn',
                'scaled_offer_value_knn',
                'estimated_expenses_knn',
                'offer_value_knn',
                'age_knn',
                'log_age_knn',
                'age'], inplace=True)

In [626]:
X.head(1)

,cc_len,cc_startswith,log_salary,log_estimated_expenses_knn,log_offer_value_knn,nan_age,not_nan_age,target,offer_class,gender,phone_calls,emails,customer_type,center,0_emails,1_email,2_emails,3_emails,4_emails,5_emails
0,ELSE,ELSE,7.329474,8.882808,5.71694,True,False,False,Premium,female,1,2,C,B,0,0,1,0,0,0


In [627]:
len(X.columns)

20

In [628]:
X.head(1)

,cc_len,cc_startswith,log_salary,log_estimated_expenses_knn,log_offer_value_knn,nan_age,not_nan_age,target,offer_class,gender,phone_calls,emails,customer_type,center,0_emails,1_email,2_emails,3_emails,4_emails,5_emails
0,ELSE,ELSE,7.329474,8.882808,5.71694,True,False,False,Premium,female,1,2,C,B,0,0,1,0,0,0


In [629]:
X.drop(columns=['emails'],inplace=True)

In [630]:
cs = [0.01]# ,0.05]#, 0.1, 0.5, 1,5]
solvers = ['liblinear']
random_states = [42]
tols = [0.01]#,0.001]#,0.0001]
max_iters = [100]#,200]
penaltys = ['l1']#, 'l2']

In [864]:
def LOO_GridSearchCV(train_df, model: str, params_grid, columns_to_encode, columns_to_drop_from_training, n_folds: int = 5):
    """
    """
    
    processed_train_df = train_df.copy(deep=True)
    
    X = processed_train_df.reset_index().drop(columns='name')
    y = processed_train_df[['target']].reset_index().drop(columns='name')
    
    email_ohe_names = {0: '0_emails',
               1: '1_email',
               2: '2_emails',
               3: '3_emails',
               4: '4_emails',
               5: '5_emails'
              }
    
    
    processed_train_df = pd.concat([processed_train_df, pd.get_dummies(processed_train_df['emails'])], axis=1).rename(columns=email_ohe_names)
    columns_to_drop_from_training.append('emails')
    X.drop(columns=columns_to_drop_from_training, inplace=True)
    
    
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=None)
    #to have each sample exactly once in validation set
    splits = kf.split(processed_train_df)
    
    dfs_to_mean = []
    Xs_train, ys_train, Xs_val, ys_val = ([] for i in range(4))
    for train_index, val_index in splits:
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_val, y_val = X.iloc[val_index], y.iloc[val_index]
        X_val.drop(columns=['target'], inplace=True)

        enc = LeaveOneOutEncoder(train_df=X_train,
                             test_df=X_val,
                             columns_to_encode=columns_to_encode,
                             target_column='target',
                             random_state=42,
                             mean=1,
                             std=0.05)
        X_train, X_val = enc.fit()
        encoded_cols = [col for col in X_train.columns if 'encoded_' in col]
        dfs_to_mean.append(X_train[encoded_cols])
        train_to_drop = columns_to_encode.copy()
        train_to_drop.extend(['target'])
        X_train.drop(columns=train_to_drop, inplace=True)
        test_to_drop = columns_to_encode.copy()
        X_val.drop(columns=test_to_drop, inplace=True)
        
        Xs_train.append(X_train)
        ys_train.append(y_train)
        Xs_val.append(X_val)
        ys_val.append(y_val)

            
        
    for df in dfs_to_mean:
        zeros = [0 for col in df.columns]
        for index in range(len(processed_train_df)):
            if index not in df.index:
                df.loc[index, :] = zeros
        df.sort_index(inplace=True)
        
    mean_df = dfs_to_mean[0].copy(deep=True)
    mean_df = mean_df*0
    for num in range(n_folds):
        mean_df = mean_df+dfs_to_mean[num]
    encoded_final_df = mean_df.divide(n_folds-1) 
 
    
    models_accuracies, models_recalls, models_parameters, models_cms = ([] for i in range(4))
    for p in itertools.product(*params_grid.values()):
        model_params = params_grid.copy()
        for counter, key in enumerate(model_params.keys()):
            model_params[key] = p[counter]
        
        models_parameters.append(model_params.items())
        if model == 'LogisticRegression':
            clf = LogisticRegression(**model_params)
            
        if model == 'RandomForestClassifier':
            clf = RandomForestClassifier(**model_params)
        
        cv_accuracies, cv_recalls, cv_cms = ([] for i in range(3))
        
        for index in range(n_folds):
            clf.fit(Xs_train[index], ys_train[index])
            predictions = clf.predict(Xs_val[index])
            cv_accuracies.append(accuracy_score(ys_val[index], predictions))
            cv_recalls.append(recall_score(ys_val[index], predictions))
            cv_cms.append(confusion_matrix(ys_val[index], predictions))

        models_accuracies.append(np.mean(cv_accuracies))
        models_recalls.append(np.mean(cv_recalls))
        models_cms.append(np.mean(cv_cms, axis=0))
        
    
    return encoded_final_df, models_accuracies, models_parameters, models_recalls, models_cms

In [722]:
#confusion_matrix-threshold

In [634]:
X_train.columns

Index(['log_salary', 'log_estimated_expenses_knn', 'log_offer_value_knn',
       'nan_age', 'not_nan_age', '0_emails', '1_email', '2_emails', '3_emails',
       '4_emails', '5_emails', 'encoded_offer_class', 'encoded_gender',
       'encoded_customer_type', 'encoded_center', 'encoded_phone_calls',
       'encoded_cc_len', 'encoded_cc_startswith'],
      dtype='object')

at the beginning, I will pick LogisticRegression, RandomForestClassifier, KNeighborsClassifier, GaussianProcessClassifier and GradientBoostingClassifier and then try to do ensemble of them by using VotingClassifier (voting='soft')

In [24]:
from ml_preprocessing.encoding_helpers import loo_grid_search_cv

In [27]:
help(loo_grid_search_cv)

Help on function loo_grid_search_cv in module ml_preprocessing.encoding_helpers:

loo_grid_search_cv(train_df, model:str, params_grid:DefaultDict, columns_to_encode:List, columns_to_drop_from_training:List, email_ohe_names:DefaultDict=None, mean:int=1, std:int=0.05, n_folds:int=5) -> Tuple[pandas.core.frame.DataFrame, List, List, List, List]
    Specially prepared function to do grid search with cross-validation on our loo encoded
    DataFrame.
    
    Scores should be approximately ok, although i have no proof for that :)
    
    :param train_df: train_df (will be splitted then to train/and_val n_folds times)
    :param model: model to train, currently supports only LogisticRegression
                  and RandomForestClassifier
    :param params_grid: param_grid to search
    :param columns_to_encode: categorical columns, which you want to encode using loo
    :param columns_to_drop_from_training: columns to drop from training phase
    :param email_ohe_names: Dictionary containin

In [30]:
params_grid = {'C': [0.01, 0.05, 0.1, 0.2, 0.5],
               'random_state': [42]}

In [31]:
email_ohe_names = {0: '0_emails',
                   1: '1_email',
                   2: '2_emails',
                   3: '3_emails',
                   4: '4_emails',
                   5: '5_emails'
                   }

In [32]:
encoded_df, accuracies, parameters,  recalls, cms = loo_grid_search_cv(
                                                     train_df=processed_train_df,
                                                     model='LogisticRegression',
                                                     params_grid=params_grid,
                                                     columns_to_encode=['offer_class', 'gender', 'customer_type',
                                                                        'center', 'phone_calls', 'cc_len', 'cc_startswith'],
                                                     columns_to_drop_from_training=['salary', 'scaled_salary',
                                                                                    'scaled_age_knn', 'scaled_estimated_expenses_knn',
                                                                                    'scaled_offer_value_knn', 'estimated_expenses_knn', 'offer_value_knn',
                                                                                    'age_knn', 'log_age_knn', 'age'],
                                                     email_ohe_names=email_ohe_names,
                                                     mean=1,
                                                     std=0.05,
                                                     n_folds=5)

In [33]:
encoded_df.head()

,encoded_offer_class,encoded_gender,encoded_customer_type,encoded_center,encoded_phone_calls,encoded_cc_len,encoded_cc_startswith
0,0.263336,0.741888,0.592548,0.399059,0.528208,0.323245,0.374871
1,0.420247,0.190426,0.315529,0.363879,0.338293,0.313231,0.363258
2,0.262989,0.743790,0.371021,0.399731,0.527031,0.324004,0.376155
3,0.276522,0.206354,0.622218,0.394315,0.366588,0.339431,0.393642
4,0.251757,0.187654,0.312258,0.382659,0.334890,0.310166,0.360089


In [866]:
encoded_df.head()

,encoded_offer_class,encoded_gender,encoded_customer_type,encoded_center,encoded_phone_calls,encoded_cc_len,encoded_cc_startswith
0,0.263336,0.741888,0.592548,0.399059,0.528208,0.323245,0.374871
1,0.420247,0.190426,0.315529,0.363879,0.338293,0.313231,0.363258
2,0.262989,0.743790,0.371021,0.399731,0.527031,0.324004,0.376155
3,0.276522,0.206354,0.622218,0.394315,0.366588,0.339431,0.393642
4,0.251757,0.187654,0.312258,0.382659,0.334890,0.310166,0.360089


In [34]:
parameters

[dict_items([('C', 0.01), ('random_state', 42)]),
 dict_items([('C', 0.05), ('random_state', 42)]),
 dict_items([('C', 0.1), ('random_state', 42)]),
 dict_items([('C', 0.2), ('random_state', 42)]),
 dict_items([('C', 0.5), ('random_state', 42)])]

In [868]:
parameters

[dict_items([('C', 0.01), ('random_state', 42)]),
 dict_items([('C', 0.05), ('random_state', 42)]),
 dict_items([('C', 0.1), ('random_state', 42)]),
 dict_items([('C', 0.2), ('random_state', 42)]),
 dict_items([('C', 0.5), ('random_state', 42)])]

In [35]:
accuracies

[0.6711833403686507,
 0.7342291637352846,
 0.7468458327470568,
 0.7701468036208432,
 0.7701468036208433]

In [867]:
accuracies

[0.6711833403686507,
 0.7342291637352846,
 0.7468458327470568,
 0.7701468036208432,
 0.7701468036208433]

In [869]:
recalls

[0.19425974025974027,
 0.4405947385947385,
 0.5358901098901099,
 0.629004995004995,
 0.6598374958374958]

In [36]:
recalls

[0.19425974025974027,
 0.4405947385947385,
 0.5358901098901099,
 0.629004995004995,
 0.6598374958374958]